# $T_1$ under repeated measurement using QI 2.0, with saving of raw data into .csv file (legacy code)

**Authors:** G. Ruggero Di Carlo (rdicarlo@tudelft.nl), Marios Samiotis (m.samiotis@tudelft.nl)

**Date:** February 14, 2025

NOTE: this notebook contains legacy code which was used with Quantum Inspire (QI) 1.0, and has now been translated to QI 2.0. Apart from the translation to QI 2.0, this code and format are not being currently supported.

In this notebook we will perform a $T_1$ measurement using custom functions from utility_functions.py, and save the raw data in a .csv file in order to be processed after measurement.

We use the mid-circuit measurement functionality for this purpose.

For optimal performance, make sure that you are running this notebook within a customized Python 3.12 environment which includes the packages "quantuminspire" and "qiskit-quantuminspire".

For detailed instructions on how to create such a Python environment, follow the instructions in the README file, [https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/README.md](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/README.md).

NOTE: in order to run the custom functions from utility_functions.py, you will need to clone the entire repository QuantumInspireUtilities from [https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/tree/main](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/tree/main).

Useful links:
1. [Tuna Backends Operational Specifics](https://www.quantum-inspire.com/kbase/tuna-operational-specifics/)
2. [Backend Performance Dashboards](https://monitoring.qutech.support/public-dashboards/c494a21fb6b7405f850ab8f340f798ef?orgId=1&refresh=10s) [live updates]
3. [Join our Slack community!](https://join.slack.com/t/qisuperconducting/shared_invite/zt-35o7zitdh-_9QPmB53hhLy12Eat5gwWA)

In [ ]:
from utility_functions import *
from qiskit import QuantumCircuit
import matplotlib.pyplot as plt

!qi login "https://api.quantum-inspire.com"

In [ ]:
experiment_backend = "Tuna-9"
provider = QIProvider()
backend = provider.get_backend(name=experiment_backend)

Using the function create_new_data_folder() will create a folder in the path specified in the utility_functions.py where we will save all our results.

In [ ]:
data_directory = f"[...]"

new_directory_path = create_new_data_folder(datadir=data_directory)

We now create the quantum circuit to perform a single-qubit $T_1$ measurement using Qiskit,

In [ ]:
nr_qubits = 9 # Number of qubits of Tuna-9 processor
nr_points = 15 # Number of points for the T1 experiment
total_time = 150e-6 # Total time of the T1 experiment in [s]

qubit_index = 0 # We choose qubit Q0

cycle_time = 20e-9 # cycle time of the Central Controller (CC) instrument
dt = total_time / nr_points # time interval between each measurement point
measurement_times = np.linspace(start = 0.0, stop = total_time, num = nr_points)

qc = QuantumCircuit(9, nr_points) # NOTE: we should put backend.num_qubits here
qc.x(qubit_index) # qubit initialization to the |1> state
qc.measure(qubit = qubit_index, cbit = 0) # initial measurement at time t=0
for time_idx in range(1, nr_points):
    qc.delay(duration = int((dt / cycle_time)), qarg = qubit_index) # delay in units of CC cycles
    qc.measure(qubit = qubit_index, cbit = time_idx)

You may visualize the circuit by running the cell below,

In [ ]:
qc.draw('mpl')

Choose the name of the files that we will create and save,

In [9]:
basename = 'Example'
suffix = 'T1_under_measurement'
histname, circuit_name, rawname = prepare_file(basename, suffix, doraw=1)

Here we specify the number of shots and run the algorithm, (careful not to exceed the max_shots allowed by the backend!)

In [10]:
nr_shots = 2**14 # Careful not to exceed the max_shots of the backend!

results = api_run_and_save(0, Qcircuit = qc, rawdata_filename = rawname, circuit_name=circuit_name,
                           histname = histname, shots=nr_shots, backend_name=experiment_backend, get_results = True,
                           get_hist_data=True, measurement_list=list((np.ones(nr_points, dtype=np.int8))), get_raw_data=True)

# Data analysis

The data are saved in a csv file in the format:

In [11]:
df = pd.read_csv(rawname+"_0.csv")
measurement_outcomes_list = np.zeros((len(df["Raw data values"]), (len(df["Raw data values"][0])-1)), dtype = np.int8)

for shot_index in range(len(df["Raw data values"])):
    for measurement_index in range(1, len(df["Raw data values"][0])):
        measurement_outcomes_list[shot_index][(measurement_index - 1)] = np.int8(df["Raw data values"][shot_index][len(df["Raw data values"][0]) - measurement_index])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3), sharey = 'row', sharex = 'col')
ax.plot(np.arange(len(np.sum(measurement_outcomes_list, 0))) * (dt * 1e6 + 1), np.sum(measurement_outcomes_list, 0) / nr_shots, '-o')

ax.set_ylim(0, 1)
ax.set_xlim(-1.5, max(np.arange(len(np.sum(measurement_outcomes_list, 0))) * (dt * 1e6 + 1)) + 1.5)

ax.set_title('$T_1$ under repeated measurement')
ax.set_ylabel('Fraction')
ax.set_xlabel('Time (us)')